Downloading...
From: https://drive.google.com/uc?id=1Zn6q2yvbs3mF0O3jfxznglixlF18R_dl
To: /content/2110446-data-science-and-data-engineering-2021.zip
156MB [00:00, 168MB/s]
Archive:  2110446-data-science-and-data-engineering-2021.zip
  inflating: PM2.5_kaggle 2021/Bangkok/Test/3H_temperature_Bangkok.csv  
  inflating: PM2.5_kaggle 2021/Bangkok/Test/3H_wind_Bangkok.csv  
  inflating: PM2.5_kaggle 2021/Bangkok/Test/Bangkok.txt  
  inflating: PM2.5_kaggle 2021/Bangkok/Train/3H_temperature_Bangkok.csv  
  inflating: PM2.5_kaggle 2021/Bangkok/Train/3H_wind_Bangkok.csv  
  inflating: PM2.5_kaggle 2021/Bangkok/Train/Bangkok.txt  
  inflating: PM2.5_kaggle 2021/Chanthaburi/Test/3H_temperature_Chanthaburi.csv  
  inflating: PM2.5_kaggle 2021/Chanthaburi/Test/3H_wind_Chanthaburi.csv  
  inflating: PM2.5_kaggle 2021/Chanthaburi/Test/Chanthaburi.txt  
  inflating: PM2.5_kaggle 2021/Chanthaburi/Train/3H_temperature_Chanthaburi.csv  
  inflating: PM2.5_kaggle 2021/Chanthaburi/Train/3H_wind_Chanthabu

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from datetime import datetime, timedelta

def DataCleaner(prv,Train_Test) :
    province = prv
    province_pm = prv
    if province.lower() == 'chiangmai' :
        province = 'Chiang Mai'
        province_pm = 'Chiang_Mai'
    elif province.lower() == 'khonkaen' :
        province = 'Khon Kaen'
        province_pm = 'Khon_Kaen'

    # temperature
    df_temp = pd.read_csv('PM2.5_kaggle 2021/'+province+'/'+Train_Test+'/3H_temperature_'+province+'.csv')
    df_temp['datetime'] = pd.to_datetime(df_temp['datetime'], format='%Y-%m-%d %H:%M:%S', errors='ignore')
    df_temp['lat'] = np.round(df_temp['lat'],4)
    df_temp['long'] = np.round(df_temp['long'],4)

    # wind
    df_wind = pd.read_csv('PM2.5_kaggle 2021/'+province+'/'+Train_Test+'/3H_wind_'+province+'.csv')
    df_wind['datetime'] = pd.to_datetime(df_wind['datetime'], format='%Y-%m-%d %H:%M:%S', errors='ignore')
    df_wind['lat'] = np.round(df_wind['lat'],4)
    df_wind['long'] = np.round(df_wind['long'],4)

    # PM2.5
    pm = open('PM2.5_kaggle 2021/'+province+'/'+Train_Test+'/' + province_pm + '.txt','r')
    df_pm = []
    for line in pm :
        if line[0] == '%' :
            continue
        else :
            row = line.split()
            hour = int(row[3])
            format_time = row[0] + "-" + row[1] + "-" + row[2] + " " + str(hour) + ":00:00"
            format_time = pd.to_datetime(format_time, format='%Y-%m-%d %H:%M:%S', errors='ignore')
            format_time = format_time + timedelta(hours=7)
            format_row = [format_time,float(row[4])]
            df_pm.append(format_row)
    pm.close()
    df_pm = pd.DataFrame(df_pm,columns=['datetime','PM2.5'])
    df_pm['datetime'] = df_pm['datetime'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))

    # merge
    df = pd.merge(df_temp, df_wind,  how='left', left_on=['datetime','lat','long'], right_on = ['datetime','lat','long'])
    df['datetime'] = df['datetime'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
    df = df.set_index('datetime').join(df_pm.set_index('datetime'), on='datetime', how='outer', sort=True)
    df.reset_index(drop=True, inplace=True)
    
    # clean Na
    df['Temp(C)'] = df['Temp(C)'].interpolate('linear')
    df['WindDir'] = df['WindDir'].interpolate('linear')
    df['Wind Speed(km/h)'] = df['Wind Speed(km/h)'].interpolate('linear')
    df['PM2.5'] = df['PM2.5'].interpolate('linear')
    df['lat'] = df['lat'].fillna(df['lat'].mean())
    df['long'] = df['long'].fillna(df['long'].mean())

    # clear outliner
    df.loc[df['PM2.5'] > 250, 'PM2.5'] = None
    df['PM2.5'].interpolate('linear')

    return df.iloc[5:,:]


In [3]:
prvs = ['Chanthaburi','Chiangmai','Kanchanaburi','Bangkok','Khonkaen','Songkhla']

for prv in prvs :
    for t in ['Train','Test'] :
        df = DataCleaner(prv,t)
        print(prv,t,list(df.columns))
        if df.isna().sum().sum() != 0 : # check na
            print(prv,t)


Chanthaburi Train ['datetime', 'Temp(C)', 'lat', 'long', 'WindDir', 'Wind Speed(km/h)', 'PM2.5']
Chanthaburi Test ['datetime', 'Temp(C)', 'lat', 'long', 'WindDir', 'Wind Speed(km/h)', 'PM2.5']
Chiangmai Train ['datetime', 'Temp(C)', 'lat', 'long', 'WindDir', 'Wind Speed(km/h)', 'PM2.5']
Chiangmai Test ['datetime', 'Temp(C)', 'lat', 'long', 'WindDir', 'Wind Speed(km/h)', 'PM2.5']
Kanchanaburi Train ['datetime', 'Temp(C)', 'lat', 'long', 'WindDir', 'Wind Speed(km/h)', 'PM2.5']
Kanchanaburi Test ['datetime', 'Temp(C)', 'lat', 'long', 'WindDir', 'Wind Speed(km/h)', 'PM2.5']
Bangkok Train ['datetime', 'Temp(C)', 'lat', 'long', 'WindDir', 'Wind Speed(km/h)', 'PM2.5']
Bangkok Test ['datetime', 'Temp(C)', 'lat', 'long', 'WindDir', 'Wind Speed(km/h)', 'PM2.5']
Khonkaen Train ['datetime', 'Temp(C)', 'lat', 'long', 'WindDir', 'Wind Speed(km/h)', 'PM2.5']
Khonkaen Train
Khonkaen Test ['datetime', 'Temp(C)', 'lat', 'long', 'WindDir', 'Wind Speed(km/h)', 'PM2.5']
Songkhla Train ['datetime', 'Temp(C)

In [4]:
df_kk = DataCleaner('Khonkaen','Train')
df_kk

,datetime,Temp(C),lat,long,WindDir,Wind Speed(km/h),PM2.5
5,2016-03-03 15:00:00,32.100000,16.4467,102.833,85.0,8.000000,35.800000
6,2016-03-03 16:00:00,33.000000,16.4467,102.833,80.0,6.000000,35.800000
7,2016-03-03 17:00:00,31.366667,16.4467,102.833,95.0,6.666667,36.000000
8,2016-03-03 18:00:00,29.733333,16.4467,102.833,110.0,7.333333,35.900000
9,2016-03-03 19:00:00,28.100000,16.4467,102.833,125.0,8.000000,40.166667
...,...,...,...,...,...,...,...
18982,2019-03-18 02:00:00,32.000000,16.4467,102.833,125.0,5.000000,78.200000
18983,2019-03-18 03:00:00,32.000000,16.4467,102.833,125.0,5.000000,80.000000
18984,2019-03-18 04:00:00,32.000000,16.4467,102.833,125.0,5.000000,85.500000
18985,2019-03-18 05:00:00,32.000000,16.4467,102.833,125.0,5.000000,88.300000


In [6]:
fmt_prvs = ['bangkok','chanthaburi','chiangmai','kanchanaburi','khonkaen','songkhla']

os.makedirs('Train')
os.makedirs('Test')

for i in range(len(fmt_prvs)) :
    for t in ['Train','Test'] :
        df = DataCleaner(prvs[i],t)
        df.to_csv(t+'/'+fmt_prvs[i]+'_'+t+'.csv', index=False)

In [9]:
!zip -r data ./Train ./Test

  adding: Train/ (stored 0%)
  adding: Train/kanchanaburi_Train.csv (deflated 87%)
  adding: Train/bangkok_Train.csv (deflated 87%)
  adding: Train/chanthaburi_Train.csv (deflated 87%)
  adding: Train/songkhla_Train.csv (deflated 87%)
  adding: Train/chiangmai_Train.csv (deflated 87%)
  adding: Train/khonkaen_Train.csv (deflated 86%)
  adding: Test/ (stored 0%)
  adding: Test/bangkok_Test.csv (deflated 87%)
  adding: Test/songkhla_Test.csv (deflated 87%)
  adding: Test/chanthaburi_Test.csv (deflated 86%)
  adding: Test/kanchanaburi_Test.csv (deflated 87%)
  adding: Test/khonkaen_Test.csv (deflated 86%)
  adding: Test/chiangmai_Test.csv (deflated 86%)
